In [ ]:
import os
import pandas as pd
import textwrap
import google.generativeai as genai
from dotenv import load_dotenv
from IPython.display import display
from IPython.display import Markdown

In [ ]:
# load the environment variables
load_dotenv()
# fetch an environment variable.
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

LLM 改寫 Query

In [ ]:
# prompt for the User imput
def make_prompt_user(query):
    prompt = ("""
                You are excel at extracting main points from a paragraph and further generating keywords for googling. 
                Your job is to discern the users' intent from the paragraph and 
                formulate keywords in both input language and English, producing at least 3 distinct results.
                These keywords are for getting most relevant results from Google search.
                Optimizing these keywords according to the rules of each different language systems.
                If users' intent is not clear, you should ask for more information.
                If the user requests information unrelated to AI tool recommendations, return 
                "Sorry, I can only search for information about AI tools".
                For information related to tutorials, append 'Medium' or other reputable journalism sites to the 
                end of the search query. Ensure that the search results of queries are relevant to the paragraph and 
                include tool tutorials. Finally, print out the finalized keywords.
                QUESTION: '{query}'
                ANSWER:
                """).format(query=query)
    return prompt

In [ ]:
# start with gemini model
model = genai.GenerativeModel('gemini-pro')

prompt = "我需要一個AI工具來幫助我安排行程。我需要詳細的教學。有什麼推薦的工具嗎？最好是免費的工具"
# prompt = "I need an AI tool to help me plan my trip. I need detailed tutorials. Any recommended tools? Preferably free tools."

response = model.generate_content(
    make_prompt_user(prompt),
    generation_config={
        'temperature': 0.5,
        'top_k': 40,
    }
)

print(response.text)

LLM 將爬蟲結果整理成 markdown 格式

In [ ]:
# libraries for web scraping
import requests
from bs4 import BeautifulSoup

In [ ]:
# prompt for turn summary of website info into markdown file
def make_prompt_mdfile(query):
    prompt = ("""
                You are good at summarizing information of the website and return the results in markdown format.
                Website_html: '{query}'
                ANSWER:
                """).format(query=query)
    return prompt

In [ ]:
# 從給定的URL抓取網頁內容
def fetch_webpage(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text

# 從HTML中解析內容
def parse_html(html):
    soup = BeautifulSoup(html, 'html.parser')

    content = ' '.join([p.text for p in soup.find_all('p')])
    content = soup.title.string + ' ' + content
    return content

def web_to_md(url):
    html = fetch_webpage(url)
    content = parse_html(html)
    response = model.generate_content(
        make_prompt_mdfile(content),
        generation_config={
            'temperature': 0.5,
            'top_k': 40,
        }
    )
    print(response.text)

In [ ]:
web_to_md('https://clickup.com/blog/ai-travel-planners/')